<a href="https://colab.research.google.com/github/anu90dl/NeuralNetwork/blob/main/CNNPraise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
#download MNIST dataset into pytorch
mean_gray = 0.1307
stddev_gray = 0.3081

transform_f = transforms.Compose([transforms.ToTensor(),
                          transforms.Normalize((mean_gray,),(stddev_gray,))])

train_data = datasets.MNIST("/content/drive/MyDrive/Mnist/",
                            train = True,
                            download = True,
                            transform = transform_f)

test_data = datasets.MNIST("/content/drive/MyDrive/Mnist/",
                            train = False,
                            download = True,
                            transform = transform_f)

In [3]:
#Train loader and test loader
batch_size = 100

train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_data,
                                           batch_size = batch_size,
                                           shuffle = False)

In [4]:
#create the NN

class NN(nn.Module):

  #init all the batch
  def __init__(self):
      super(NN,self).__init__()
      #padding needed = (kernelsize - 1)/2 , (3-1)/2
      self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
      self.batchnorm1 = nn.BatchNorm2d(8)
      self.relu = nn.ReLU()
      self.maxpool1 = nn.MaxPool2d(kernel_size = 2)
      # after maxpool the size will be inputsize/2 (28/2) = 14
      self.conv2 = nn.Conv2d(in_channels=8,out_channels = 32, kernel_size = 5, stride =1, padding = 2)
      self.batchnorm2 = nn.BatchNorm2d(32)
      # after maxpool output will be inputsize/2 (14/2)= 7
      # the layer will be 7*7*32 = 1568
      self.fc1 = nn.Linear(1568,600)
      self.droput = nn.Dropout(p=0.5)
      self.fc2 = nn.Linear(600,10)

  def forward(self,x):
      out = self.conv1(x)
      out = self.batchnorm1(out)
      out = self.relu(out)
      out = self.maxpool1(out)
      out = self.conv2(out)
      out = self.batchnorm2(out)
      out = self.relu(out)
      out = self.maxpool1(out)
      out = out.view(-1,1568)
      out = self.fc1(out)
      out = self.relu(out)
      out = self.droput(out)
      out = self.fc2(out)

      return out


In [5]:
#create the loss and optimizer function
model = NN()
CUDA = torch.cuda.is_available()

if CUDA:
  model = model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.1)

In [6]:
# training iterations
train_loss = []
test_loss = []
train_correct = []
test_correct = []
epochs = 10

for epoc in range(epochs):
  iterations = 0
  iter_loss = 0.0
  correct = 0
  test_iterations = 0
  test_iter_loss = 0
  t_correct = 0


  for i,(inputs,labels) in enumerate(train_loader):
    if CUDA:
      inputs = inputs.cuda()
      labels = labels.cuda()
      # run the input through the model, batch training (batch size 100)
    outputs = model(inputs)
    # take the predictions
    _, predicted = torch.max(outputs,1)
    #check the accuracy
    correct += (predicted == labels).sum().item()

    # calculate the loss
    loss = loss_fn(outputs,labels)
    #this iteration loss
    iter_loss += loss.item()
    # clear the gradient
    optimizer.zero_grad()
    #backpropagate
    loss.backward()
    #calculate the new weights
    optimizer.step()

    #count iterations
    iterations += 1

  train_loss.append(iter_loss/iterations)
  train_correct.append((correct/len(train_loader)))

# put model into evaluation mode
  model.eval()

# check the testing loss
  for i,(test_inputs,test_labels) in enumerate(test_loader):
    if CUDA:
      test_inputs = test_inputs.cuda()
      test_labels = test_labels.cuda()

    test_outputs = model(test_inputs)
    _, test_predictions = torch.max(test_outputs,1)
    t_correct += (test_predictions == test_labels).sum().item()
    test_iter_loss += loss_fn(test_outputs,test_labels)

    test_iterations += 1

  test_loss.append(test_iter_loss/test_iterations)
  test_correct.append((t_correct/ len(test_loader)))


  print("Epochs - {}/{} , train loss-{:.3f}, train accuracy-{:.3f},test loss-{:.3f},test accuracy-{:.3f}".format(epoc +1 ,epochs,train_loss[-1],train_correct[-1],test_loss[-1],test_correct[-1]))



Epochs - 1/10 , train loss-0.199, train accuracy-93.675,test loss-0.050,test accuracy-98.380
Epochs - 2/10 , train loss-0.167, train accuracy-95.587,test loss-0.051,test accuracy-98.210
Epochs - 3/10 , train loss-0.040, train accuracy-98.753,test loss-0.032,test accuracy-98.860
Epochs - 4/10 , train loss-0.027, train accuracy-99.158,test loss-0.037,test accuracy-98.790
Epochs - 5/10 , train loss-0.019, train accuracy-99.388,test loss-0.037,test accuracy-98.910
Epochs - 6/10 , train loss-0.014, train accuracy-99.558,test loss-0.033,test accuracy-98.990
Epochs - 7/10 , train loss-0.011, train accuracy-99.650,test loss-0.032,test accuracy-98.960
Epochs - 8/10 , train loss-0.008, train accuracy-99.725,test loss-0.036,test accuracy-98.940
Epochs - 9/10 , train loss-0.006, train accuracy-99.820,test loss-0.031,test accuracy-99.080
Epochs - 10/10 , train loss-0.004, train accuracy-99.898,test loss-0.030,test accuracy-99.260
